In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [2]:
#Need OHE for better results

In [3]:
X_train = pd.read_csv('./data/processed/X_train.csv', na_filter=False)
X_val = pd.read_csv('./data/processed/X_val.csv', na_filter=False)
X_test = pd.read_csv('./data/processed/X_test.csv', na_filter=False)

y_train = pd.read_csv('./data/processed/y_train.csv', na_filter=False)
y_val = pd.read_csv('./data/processed/y_val.csv', na_filter=False)
y_test = pd.read_csv('./data/processed/y_test.csv', na_filter=False)

In [4]:

from sklearn.preprocessing import OneHotEncoder
feature_names = X_train.columns
dataset = pd.concat([X_train, X_test])
numeric_features = ['YearsCode', 'YearsCodePro', 'Age', 'Age1stCode']
categorical_features = list(filter(lambda col: col not in numeric_features, X_train.columns))
categorical_features_indices = list(filter(lambda idx: idx not in [4, 5], list(range(51))))

dataset = dataset[feature_names].values
val_dataset = X_val.copy()
val_dataset = val_dataset[feature_names].values
train_dataset = X_train.copy()
train_dataset = train_dataset[feature_names].values
test_dataset = X_test.copy()
test_dataset = test_dataset[feature_names].values
categorical_names = {} 
les={} 
for feature in categorical_features_indices:
    le = LabelEncoder() 
    le.fit(dataset[:, feature])
    dataset[:, feature] = le.transform(dataset[:, feature])
    val_dataset[:, feature] = le.transform(val_dataset[:, feature])
    train_dataset[:, feature] = le.transform(train_dataset[:, feature])
    test_dataset[:, feature] = le.transform(test_dataset[:, feature])
    categorical_names[feature] = le.classes_ 
    les[feature] = le

dataset = dataset.astype('float32') 
val_dataset = val_dataset.astype('float32') 
train_dataset = train_dataset.astype('float32') 
test_dataset = test_dataset.astype('float32')
from sklearn.compose import ColumnTransformer 
encoder = ColumnTransformer([("enc", OneHotEncoder(),
                            categorical_features_indices)],
                            remainder = 'passthrough')

encoder.fit(dataset)
 
encoded_dataset = encoder.transform(dataset)
encoded_train = encoder.transform(train_dataset)
encoded_val = encoder.transform(val_dataset)
encoded_test = encoder.transform(test_dataset)

In [5]:
import pandas
from sklearn.linear_model import Lasso

In [ ]:
lasso_model = Lasso()
lasso_model.fit(encoded_train, y_train)
print(mean_absolute_error(lasso_model.predict(encoded_test), y_test))

In [ ]:
import lime
import lime.lime_tabular
from lime import submodular_pick
import sklearn
from sklearn.preprocessing import LabelEncoder

In [ ]:
labels = pd.concat([y_train, y_test])
le= LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
#dataset = dataset[feature_names].values

In [ ]:
#categorical_names = {} 
les={} 
for feature in categorical_features_indices:
    le = LabelEncoder() 
    le.fit(dataset[:, feature])
    dataset[:, feature] = le.transform(dataset[:, feature])
    #categorical_names[feature] = le.classes_ 
    les[feature] = le

dataset = dataset.astype(float)

In [ ]:
train = dataset[:X_train.shape[0], :]
test = dataset[X_train.shape[0]:, :]
labels_train = labels[:X_train.shape[0]]
labels_test = labels[X_train.shape[0]:]

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train,mode='regression',feature_names=feature_names,
                                                   categorical_features=categorical_features_indices,
                                                   categorical_names=categorical_names,
                                                   class_names=['ConvertedCompYearly'])

In [ ]:
predict_fn = lambda x: lasso_model.predict(np.asarray(encoder.transform(x)).astype('float32')).astype(float)

In [ ]:
n_explanations = 20

In [ ]:
diff_array = pd.DataFrame() 
diff_array['real'] = pd.concat([y_test])
diff_array['predict'] = lasso_model.predict(encoded_test)
diff_array['diff'] = diff_array['real'] - diff_array['predict']


In [ ]:
small_but_big_predict =3545    
big_but_small_predict =6217
big_and_big_predict = 8088    
small_and_small_predict=8140 

In [ ]:
exp = explainer.explain_instance(test[big_and_big_predict], predict_fn, num_features=n_explanations)
print(f"""Real       ConvertedCompYearly= {diff_array.iloc[[big_and_big_predict]]['real'].values}
Predicted  ConvertedCompYearly= {diff_array.iloc[[big_and_big_predict]]['predict'].values}""")
exp.show_in_notebook(show_all=False)

In [ ]:
exp = explainer.explain_instance(test[small_and_small_predict], predict_fn, num_features=n_explanations)
print(f"""Real       ConvertedCompYearly= {diff_array.iloc[[small_and_small_predict]]['real'].values}
Predicted  ConvertedCompYearly= {diff_array.iloc[[small_and_small_predict]]['predict'].values}""")
exp.show_in_notebook(show_all=False)

In [ ]:
exp = explainer.explain_instance(test[small_but_big_predict], predict_fn, num_features=n_explanations)
print(f"""Real       ConvertedCompYearly= {diff_array.iloc[[small_but_big_predict]]['real'].values}
Predicted  ConvertedCompYearly= {diff_array.iloc[[small_but_big_predict]]['predict'].values}""")
exp.show_in_notebook(show_all=False)

In [ ]:
exp = explainer.explain_instance(test[big_but_small_predict], predict_fn, num_features=n_explanations)
print(f"""Real       ConvertedCompYearly= {diff_array.iloc[[big_but_small_predict]]['real'].values}
Predicted  ConvertedCompYearly= {diff_array.iloc[[big_but_small_predict]]['predict'].values}""")
exp.show_in_notebook(show_all=False)